In [1]:
from itertools import combinations
from collections import defaultdict
import docker
from numpy import sign
import re

In [62]:
def extract_sol(s):
    digitfinder = re.compile("-*\d+")
    find = digitfinder.findall(s)
    digits = set(map(int, find))
    if 0 in digits:
        digits.remove(0)
    return digits
def find_solutions():
    client = docker.from_env()
    maxsol = 1000
    solve_problem_cmd = "cryptominisat5 --verb 0 --maxsol {} /data/problem2.cnf".format(maxsol)
    _,out = client.containers.get("cms").exec_run(solve_problem_cmd)
    getsol = lambda sol: {sign(elt)*elt: elt>0 for elt in sol}
    all_lines = iter(out.decode("utf-8").split("\n"))
    line = next(all_lines)
    s = []
    while not (line == "s UNSATISFIABLE"):
        if line == "s SATISFIABLE":
            s.append({})
        else:
            #print("|{}|".format(line))
            s[-1].update(getsol(extract_sol(line)))
        line = next(all_lines)
    return s

In [27]:
def list2filestr(cnf):
    return "\n".join([" ".join(map(str, lnf)) for lnf in cnf])

def write2cnf(start, cnf):
    path = "C:/Users/johan/PycharmProjects/sat/problem2.cnf"
    with open(path, "w") as text_file:
            cnfstr = "p cnf {} {}\n".format(start, len(cnf))
            cnfstr += list2filestr(cnf)
            text_file.write(cnfstr)

In [41]:
out

[{3: True, 2: False, 4: False, 1: False},
 {2: True, 4: True, 3: False, 1: False}]

In [65]:
sets = (
    {1, 2, 3},
    {4, 5},
    {6, 7},
    {2, 4},
    {3, 5, 6},
)

In [66]:
linecnf = sets[0].union(*sets[1:])
start = len(linecnf)

In [67]:
def combocnf(combo):
    return {(-c1, -c2, 0) for c1, c2 in combinations(combo,2)}

In [68]:
cnf = {elt for combo in sets for elt in combocnf(combo)}
for s in sets:
    cnf.add(tuple(s)+(0,))

In [69]:
cnf

{(-6, -7, 0),
 (-5, -6, 0),
 (-4, -5, 0),
 (-3, -6, 0),
 (-3, -5, 0),
 (-2, -4, 0),
 (-2, -3, 0),
 (-1, -3, 0),
 (-1, -2, 0),
 (1, 2, 3, 0),
 (2, 4, 0),
 (3, 5, 6, 0),
 (4, 5, 0),
 (6, 7, 0)}

In [70]:
write2cnf(start, cnf)

In [71]:
out = find_solutions()

In [72]:
out

[{2: True, 5: True, 7: True, 6: False, 4: False, 3: False, 1: False},
 {3: True, 4: True, 7: True, 2: False, 6: False, 5: False, 1: False},
 {1: True, 4: True, 6: True, 7: False, 5: False, 3: False, 2: False}]